# Set up Traffic Generator Code

In [22]:
import binascii
import random
import struct
import numpy as np
import os

from itertools import izip

PACKET_HEADER_FMT = "<II4BIi"
PACKET_HEADER_LEN = struct.calcsize(PACKET_HEADER_FMT)

FRAME_HEADER_FMT = "<II"
FRAME_HEADER_LEN = struct.calcsize(FRAME_HEADER_FMT)



def make_random_packet(preamble, sync, payload_len, counter):
    '''
    preamble and sync are expected to be bytearray or bytes objects
    
    payload len should not exceed 255
    '''
    
    if (payload_len > 255-PACKET_HEADER_LEN) or (payload_len < 0):
        # payload len is an 8 bit field so cannot be larger than 255
        raise ValueError("Payload len must be >=0 and < {}".format(256-PACKET_HEADER_LEN))
    
    # counter takes up 2 bytes
    payload = os.urandom(payload_len-2)
    
    packet_len = len(payload) + PACKET_HEADER_LEN
    
    # assemble fields to compute CRC32
    packed_fields = struct.pack(PACKET_HEADER_FMT, preamble,
                                                   sync,
                                                   packet_len, packet_len, 
                                                   packet_len, packet_len,
                                                   counter,
                                                   0)
    
    # only compute CRC for fields up to the crc itself
    crc = binascii.crc32(packed_fields[:-4])
    
    # now insert CRC
    packed_fields = struct.pack(PACKET_HEADER_FMT, preamble,
                                                   sync,
                                                   packet_len, packet_len, 
                                                   packet_len, packet_len,
                                                   counter,
                                                   crc)
    
    
    return packed_fields + payload


def make_frame(pre_spacing, packet):
    '''
    rework this to remove need for frame_count, num bursts, bytes_per_sym
    '''
    
    frame_len = len(packet) + FRAME_HEADER_LEN
    
    frame_header = struct.pack(FRAME_HEADER_FMT, 
                               pre_spacing,
                               frame_len)

    
    return frame_header + packet


def make_random_data_file(preamble, sync, max_spacing, num_bits, truth_name, out_name):
    
    # setup constants
    syms_per_byte = 4
    samps_per_sym = 4
    
    samps_per_byte = samps_per_sym*syms_per_byte
    generated_bits = 0
    packet_sizes = []
    while generated_bits < num_bits:
        packet_size = np.random.randint(low=2,high=256/4-PACKET_HEADER_LEN, size=1)[0]
        packet_sizes.append(packet_size)
        generated_bits += packet_size*8

    num_packets = len(packet_sizes)
    print("generating {} bits in {} packets".format(sum(packet_sizes)*8, num_packets))
    
    pre_spacing = np.random.randint(low=samps_per_byte*160,high=max_spacing, size=(num_packets,))
    #post_spacing = np.random.randint(low=1,high=max_spacing, size=(num_packets,))
    
    

    #pre_spacing = [80,]*len(packet_sizes)
    #packet_sizes = [24,]*len(packet_sizes)
    # make all the packets 
    packets = [ make_random_packet(preamble, sync, packet_sizes[i], i) for i in range(num_packets)]
    

    
    # write out the packets as ground truth
    with open(truth_name, 'wb') as f:
        for p in packets:
            f.write(p)
            
    
    # encapsulate packets in frames and write to disk
    with open(out_name, 'wb') as f:
        for pre, packet in izip(pre_spacing, packets):
            frame = make_frame(pre, packet)
            f.write(frame)

# Configure traffic generator and write file to disk

In [23]:
preamble = 0x99999999
sync = 0x1DFCCF1A
max_spacing = 256*8*2
num_bits = 1e6

truth_name = "/home/cpomeroy/Projects/DARPA/hurdle1/truth_packets.bin"
out_name = "/home/cpomeroy/Projects/DARPA/hurdle1/py_packets.bin"

make_random_data_file(preamble, sync, max_spacing, num_bits, truth_name, out_name)


generating 1000152 bits in 5570 packets


# Generate IQ Samples

In [ ]:
from matlab.hurdle_1 import hurdle_1
tb = hurdle_1()
tb.start()
tb.wait()

# Set up code to parse decoded packets

In [24]:
def parse_packets(preamble, sync, packet_bytes):
    sync_header = struct.pack("<II", preamble, sync)
    
    packets = []
    pos = packet_bytes.find(sync_header)
    
    while pos>=0:
        
        next_pos = packet_bytes.find(sync_header, pos+1)
        
        if next_pos > 0:
            packets.append(packet_bytes[pos:next_pos])
        else:
            packets.append(packet_bytes[pos:])
                
        pos = next_pos
    
    
    return packets


# read in truth packets, whole file
with open(truth_name, 'rb') as f:
    truth_bytes = f.read()

# read in decoded file, whole file
decoded_name = '/home/cpomeroy/Projects/DARPA/hurdle1/matlab/decoded.bin'
with open(decoded_name, 'rb') as f:
    decoded_bytes = f.read()

truth_packets =  parse_packets(preamble, sync, truth_bytes)     
decoded_packets =  parse_packets(preamble, sync, decoded_bytes)    





# Do Naive Implementation of BER first

In [25]:
from collections import namedtuple

# test for sane packet lengths


truth_packet_counters = [struct.unpack("<I",p[12:16])[0] for p in truth_packets]
truth_packet_crcs = [struct.unpack("<i",p[16:20])[0] for p in truth_packets]
decoded_packet_counters =  [struct.unpack("<I",p[12:16])[0] for p in decoded_packets]
decoded_packet_crcs =  [struct.unpack("<i",p[16:20])[0] for p in decoded_packets]
#print(decoded_packet_counters)


correct_count = 0

for i, c in enumerate(decoded_packet_counters):
    # try to match each decoded packet counter c against a truth packet counter t
    matched_ind = next( (i for i,t in enumerate(truth_packet_counters) if t == c), -1)
    
    #print(matched_ind)
    if matched_ind >= 0:
        t_crc = truth_packet_crcs[matched_ind]
        d_crc = decoded_packet_crcs[i]
        
        # check that CRCs match
        
        if t_crc == d_crc:
        
            tp = bytearray(truth_packets[matched_ind])
            dp = bytearray(decoded_packets[i])

            tp = tp[:-8]
            dp = dp[:-8]
            
            bit_errors = sum([bin(a^b).count("1") for a,b in izip(tp, dp)])
            bits_correct = 8*len(tp)-bit_errors
            
            if bit_errors > 0:
                print("Packet found with {} bit errors. Bits correct was {}".format(bit_errors, bits_correct))
                print("truth:  ",binascii.hexlify(tp))
                print("decoded:",binascii.hexlify(dp))
            else:
                print("packet correct")
        else:
            print("crc fail")
            
    else:
        bit_errors = 8*len(decoded_packets[i])
        bits_correct = 0
        print("No packet found, adding {} bit errors".format(bit_errors))
        
    correct_count += bits_correct
        
bit_count = 8*sum(len(t) for t in truth_packets)        
print("found {} packets out of total {}".format(len(decoded_packets), len(truth_packets)))
ber = (bit_count-correct_count)/float(bit_count)
print("Bit error rate was {}, correct bits: {} total bits {}".format(ber, correct_count, bit_count))

crc fail
packet correct
No packet found, adding 208 bit errors
packet correct
packet correct
packet correct
packet correct
packet correct
Packet found with 15 bit errors. Bits correct was 257
('truth:  ', '999999991acffc1d2a2a2a2a09000000b81956ff615ff912a1b82ce68bf5d48d921a')
('decoded:', '999999991acffc1d2a2a2a2a09000000b81956ff615ff912a1982e6689f4c4ccdb8f')
packet correct
packet correct
packet correct
packet correct
packet correct
packet correct
packet correct
crc fail
packet correct
crc fail
crc fail
packet correct
packet correct
packet correct
packet correct
crc fail
Packet found with 7 bit errors. Bits correct was 265
('truth:  ', '999999991acffc1d2a2a2a2a1a000000c9e1fabd3026c2cae1bf8300f99396d507e3')
('decoded:', '999999991acffc1d2a2a2a2a1a000000c9e1fabd3026c2cae1bf8300d99996c52773')
packet correct
packet correct
packet correct
packet correct
crc fail
packet correct
packet correct
packet correct
No packet found, adding 272 bit errors
packet correct
packet correct
crc fail
packet 

In [51]:

sanity_checks = [ {"ind":i, "packet":p, "packet_len":len(p)} for i,p in enumerate(decoded_packets)]
packet_counters = np.array( [struct.unpack("<H",p[12:14])[0] for p in decoded_packets])


print(packet_counters)
print(np.diff(packet_counters))

for s in sanity_checks:
    p=s["packet"]
    packet_lens = np.array(struct.unpack("<BBBB",p[8:12]), dtype=np.uint8)
    unique_vals, unique_counts = np.unique(packet_lens, return_counts=True)
    
    if np.any( unique_vals==len(p) ):
        s["packet_len_is_sane"]=True
        
    #s["packet_count"]

[]
[]


In [38]:
# first pass, remove all packets that matched perfectly
matched_truth_packets = []
missing_truth_packets = []

for tp in truth_packets:
    ind, matched = next( ((i, p) for i,p in enumerate(decoded_packets) if p == tp), (-1, None))
    
    if ind>=0:
        matched_truth_packets.append(tp)
        del decoded_packets[ind] 
    else:
        missing_truth_packets.append(tp)

print("found {} matches".format(len(matched_truth_packets)))
print("missing {} packets".format(len(missing_truth_packets)))
print("{} decoded packets remain".format(len(decoded_packets)))

found 0 matches
missing 5589 packets
0 decoded packets remain


In [ ]:
max_errors_per_packet = 8

dp_ind = 0

matched_truth_packets = {}
missing_truth_packets = {}
matched_decoded_packets = {}
missing_decoded_packets = {}

match_scores = np.array((len(truth_packets), len(decoded_packets)))

# for each truth packet, store the index to the matching decoded packet
truth_decoded_inds = np.array( -1*ones(len(truth_packets),))

for tp_ind, tp in enumerate(truth_packets):
    ind, matched = next( ((i, p) for i,p in enumerate(decoded_packets) if p == tp), (-1, None))
    
    # the first packet with a perfect match wins
    if ind>=0:
        truth_decoded_inds[tp_ind] = ind

        
# now compute all the bit errors         
for i in len(truth_packets):
    for j in len(decoded_packets):
        
        # check if packets have the same length
        if len(tp) > len(dp):
            tp_check = tp[:len(dp)]
            dp_check = dp
            
        elif len(dp) > len(tp):
            tp_check = tp
            dp_check = dp[:len(tp)]
        else:
            tp_check = tp
            dp_check = dp
    
        # take xor of now matching length packets to determine match score
        packet_diff = tp_check ^ dp_check
        bit_errors = bin(packet_diff).count("1")
        
        match_scores[i,j] = bit_errors

for i in len(truth_packets):        
    
    last_match = -1
    
    # skip any packets with a perfect match
    if truth_decoded_inds[i]>=0:
        pass
    else:
        scores = match_scores[i,last_match+1:]
        print(scores)
        
        best_ind = np.argmin(scores)
        
        
            

    

In [ ]:
    ind, matched = next( ((i, p) for i,p in enumerate(decoded_packets) if p == tp), (-1, None))
    
    if ind>=0:
        matched_truth_packets.append(tp)
        del decoded_packets[ind] 
    else:
        missing_truth_packets.append(tp)

for tp_ind, tp in enumerate(truth_packets):
    sync = False
    
    while sync == False:
        # try to sync to current packet. Find current difference in packet bytes.
        # start by getting a copy of dp
        dp = decoded_packets[dp_ind]

        # check if packets have the same length
        if len(tp) > len(dp):
            tp_check = tp[:len(dp)]
            dp_check = dp
        elif len(dp) > len(tp):
            tp_check = tp
            dp_check = dp[:len(tp)]
        else:
            tp_check = tp
            dp_check = dp


        

        if bit_errors > max_errors_per_packet:
            # no sync
            
    
    pos = decoded_bytes.find(tp)
    if pos >=0:
        matched_packets.append(tp)
        decoded_bytes[pos:pos+len(tp)] = b''
    else:
        missing_packets = 

In [ ]:
a = 0x999999991acffc1d222222220000164f65bf68c800604d8dc3c4195384ef9697db59
b = 0x999999991acffc1d222222220000164f65bf68c800604d8dc3c4195384ef9697db50

abdiff = a^b
print(bin(abdiff).count("1"))
    

In [3]:
truth_name = "/home/cpomeroy/Projects/DARPA/hurdle1/truth_packets.bin"

In [ ]:
256/4

In [ ]:
260/4

In [ ]:
(256-PACKET_HEADER_LEN-FRAME_HEADER_LEN)/4.0

In [ ]:
57 + PACKET_HEADER_LEN - 2

In [ ]:
69*4

In [ ]:
PACKET_HEADER_LEN

In [ ]:
FRAME_HEADER_LEN

In [ ]:
255-14

In [ ]:
256/4.0